# Scipy vs our code

The `scipy` Gaussian kernel density estimation method is [documented here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html#scipy.stats.gaussian_kde).  From reading the source code, and comparing with ["Density Estimation for Statistics and Data Analysis" by Silverman](https://books.google.co.uk/books?id=e-xsrjsL7WkC&lpg=PR9&ots=iwStts0DZr&dq=density%20estimation%20for%20statistics%20and%20data%20analysis&lr&pg=PP1#v=onepage&q=density%20estimation%20for%20statistics%20and%20data%20analysis&f=false) see equation 4.7 for example, the method is as follows.

- Input is $(X_i)_{i=1}^n$ points in a $d$-dimensional space, say each $X_i = (X^{(i)}_j)_{j=1}^d$
- Compute the covariance matrix from this sample:
$$ S_{j,k} = \frac{1}{n-1} \sum_i (X^{(i)}_j - \overline{X}_j)(X^{(i)}_k - \overline{X}_k) $$
where $\overline{X}_j = \frac{1}{n} \sum_i X^{(i)}_j$ is the sample mean of the $j$th coordinate.
- Letting $S^{-1}$ be the inverse matrix and $|S|$ the determinate of $S$, choose $h$, a "bandwidth", and $K$, a "symmetric kernel".  Then our density estimation is
$$ f(x) = \frac{1}{|S|^{1/2} h^d n} \sum_{i=1}^n K\big( h^{-2}  (X_i-x)^t S^{-1} (X_i-x) \big) $$
- For example, the Gaussian kernel gives $K(t) = (2\pi)^{-d/2} \exp(-t/2)$
- As stated by Silverman, this procedure is equivalent to applying a linear transformation to the input data so that it has unit covariance, forming the standard KDE, and then transforming back.
- The bandwidth can be chosen by eye, or we can use a "rule of thumb".  Here we follow `scipy` and use Scott's rule, which is $h = n^{-1/(4+d)}$.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import scipy.linalg

In [2]:
class GaussianKernel():
    """Expect input as array of shape `(d,N)` of `N` samples in `d`-dimensional space."""
    def __init__(self, data, bandwidth=None):
        self.data = np.asarray(data)
        if len(self.data.shape) == 0:
            raise ValueError("Cannot be a scalar")
        if len(self.data.shape) == 1:
            self.data = self.data[None,:]
        
        self.space_dims, self.num_data_points = self.data.shape
        if bandwidth is None:
            bandwidth = self._scott()
        
        self.covariance = np.atleast_2d(np.cov(data, rowvar=1, bias=False))
        self.inv_cov = scipy.linalg.inv(self.covariance) / (bandwidth**2)
        cdet = np.sqrt(scipy.linalg.det(self.covariance))
        
        self.normalisation = cdet * (2 * np.pi * bandwidth * bandwidth) ** (self.space_dims/2)
        self.normalisation *= self.num_data_points
        
    def _scott(self):
        return self.num_data_points ** (-1 / (4 + self.space_dims))
        
    def __call__(self, t):
        t = np.asarray(t)
        if len(t.shape) == 0:
            if self.space_dims != 1:
                raise ValueError("Expect {} dimensional input".format(space_dims))
            t = t[None]
        if len(t.shape) == 1:
            t = t[None,:]

        x = self.data[:,:,None] - t[:,None,:]
        x = np.sum(x * np.sum(self.inv_cov[:,:,None,None] * x[:,None,:,:], axis=0), axis=0)

        return np.sum(np.exp(-x / 2), axis=0) / self.normalisation

In [3]:
data = np.random.random(size=(2,20))
k = GaussianKernel(data)
kernel = scipy.stats.kde.gaussian_kde(data, bw_method="scott")

In [4]:
x = np.random.random(size=(2,100))
np.testing.assert_allclose(kernel(x), k(x))

In [5]:
data = np.random.random(size=20)
k = GaussianKernel(data)
kernel = scipy.stats.kde.gaussian_kde(data, bw_method="scott")

In [6]:
x = np.random.random(size=100)
np.testing.assert_allclose(kernel(x), k(x))

In [7]:
data = np.random.random(size=(3,20))
k = GaussianKernel(data)
kernel = scipy.stats.kde.gaussian_kde(data, bw_method="scott")

x = np.random.random(size=(3,100))
np.testing.assert_allclose(kernel(x), k(x))

# Stocastic declusting

To convert the "stocastic declusting" algorithm into a more "EM-like" algorithm, I would like to do the following:

- Start with data $(X_i)_{i=1}^m$ and a probability distribution on $\{1,2,\cdots,m\}$.
- Using the probability distribution, sample from the data, giving $(Y_i)_{i=1}^n$ say.
- Form the density estimate $f$ using $(Y_i)$.
- Thus $f$ is a random variable, and it makes sense to ask what the expectation of $f$ is (with respect to our probability distribution on $\{1,2,\cdots,m\}$).